# Library import

In [ ]:
""" os """
import os

""" torch """
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler
from torch.utils.data.sampler import SubsetRandomSampler


"""tensor board"""
import torchvision
#from torch.utils.tensorboard import SummaryWriter


"""glob"""
from glob import glob

""" tqdm """
import time
from tqdm import tqdm

"""Pandas"""
import pandas as pd

""" numpy """
import numpy as np
from numpy import argmax
from PIL import Image

"""JSON"""
import json

"""sklearn"""
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error, r2_score

"""seaborn"""
import seaborn as sns

"""scipy"""
from scipy import io
from scipy import signal
from scipy.fft import fft, ifft,fftfreq
from scipy import stats

"""SUMMARY"""
from torchsummary import summary

"""time"""
import time


"""pingouin"""
import pingouin as pg

import re
import shutil
import random
import matplotlib.pyplot as plt
import scipy

# Path init

In [ ]:
os.listdir()

In [ ]:
cur_path = "path"
os.chdir(cur_path)
os.listdir()

In [ ]:
experiment_num = 
fig_save_path = 'path'

In [ ]:
folder_path = fig_save_path + str(experiment_num)

try:
    if not(os.path.isdir(folder_path)):
        os.makedirs(os.path.join(folder_path))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Failed to create directory!!!!!")
        raise

# Hyper parameters

In [ ]:
seed = 1

#validation ratio
validation_ratio = 0.1

#learning rate
lr = 0.001
 

momentum = 0.5


batch_size = 512    
test_batch_size = 512


epochs = 50
no_cuda = False

log_interval = 5

# Set the seed and GPU

In [ ]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers':0,'pin_memory':True} if use_cuda else {}

# Model

In [ ]:
class ECG_CNN(nn.Module):
    def __init__(self, num_classes=2):
        super(ECG_CNN, self).__init__()

        """Convolution"""
        self.conv1 = nn.Conv1d(12, 48, kernel_size=5, stride=1 )
        self.conv2 = nn.Conv1d(48, 96, kernel_size=5, stride=1 )
        self.conv3 = nn.Conv1d(96, 192, kernel_size=5, stride=1 )
        
        """BatchNormalize"""
        self.bn1 = nn.BatchNorm1d(48)
        self.bn2 = nn.BatchNorm1d(96)
        self.bn3 = nn.BatchNorm1d(192)
        
        """ReLU"""
        self.relu = nn.ReLU(inplace=True)
        
        """Sigmoid"""
        self.sigmoid = nn.Sigmoid()
        
        """ MaxPooling"""     
        self.maxpool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        """ Global Max, Average Pooling"""
        self.globalavrpool1 = nn.AvgPool1d(kernel_size = 621, stride=1)
        
        """Drop out"""
        self.dp = nn.Dropout(p=0.5)
        
        
        """Fully Connected """ 
        self.fc1 = nn.Linear(1*192, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_classes) 
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        
        x = self.globalavrpool1(x)
        
        
        """Fully Connected"""
        x = x.view(-1, 1*192*1) .
        
        
        # Dense
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        
        return x

# Data Path(12 Lead)

In [ ]:
with open('path', 'r') as f:
    label_data = json.load(f)

In [ ]:
len(label_data)

In [ ]:
label_data[0]

In [ ]:
data_paths_infant,data_paths_10s,data_paths_20s,data_paths_30s,data_paths_40s,data_paths_50s,data_paths_60s  = [],[],[],[],[],[],[]
data_paths_70s, data_paths_80s, data_paths_90s = [],[],[]

for i in range(0,len(label_data),1):
    
    """get label"""
    tempt = label_data[i]['age']
    
    if tempt<18:
        data_paths_infant.append(i)
    elif 18 <= tempt < 20:
        data_paths_10s.append(i)
    elif 20 <= tempt < 30:
        data_paths_20s.append(i)
    elif 30 <= tempt < 40:
        data_paths_30s.append(i)
    elif 40 <= tempt < 50:
        data_paths_40s.append(i)
    elif 50 <= tempt < 60:
        data_paths_50s.append(i)
    elif 60 <= tempt < 70:
        data_paths_60s.append(i)
    elif 70 <= tempt < 80:
        data_paths_70s.append(i)
    elif 80 <= tempt < 90:
        data_paths_80s.append(i)
    else:
        data_paths_90s.append(i)

# train, validatin, test set

In [ ]:
def Make_train_validation_test_paths(data_paths, train_ratio, valid_ratio):
    
    """count number of data paths"""
    num_datapaths = len(data_paths)
    indices = list(range(num_datapaths))
    
    train_split = int(np.floor(train_ratio * num_datapaths))
    valid_split = int(np.floor((train_ratio+valid_ratio) * num_datapaths))

    """set the seed and shuffle"""
    np.random.seed(seed)
    np.random.shuffle(indices)
    
    """spllit data paths"""
    train_data_paths      = []
    validation_data_paths = []
    test_data_pahts       = []
    
    
    train_idx, valid_idx, test_idx = indices[:train_split], indices[train_split:valid_split], indices[valid_split:]
    

    for i in range(0,len(train_idx),1):
        train_data_paths.append( data_paths[train_idx[i]])
        
    for i in range(0,len(valid_idx),1):
        validation_data_paths.append( data_paths[valid_idx[i]])
        
    for i in range(0,len(test_idx),1):
        test_data_pahts.append( data_paths[test_idx[i]])
        
    
    
    return train_data_paths, validation_data_paths, test_data_pahts

In [ ]:
train_paths_10s, validation_paths_10s, test_paths_10s = Make_train_validation_test_paths(data_paths_10s,0.4,0.3)
train_paths_20s, validation_paths_20s, test_paths_20s = Make_train_validation_test_paths(data_paths_20s,0.4,0.3)
train_paths_30s, validation_paths_30s, test_paths_30s = Make_train_validation_test_paths(data_paths_30s,0.4,0.3)
train_paths_40s, validation_paths_40s, test_paths_40s = Make_train_validation_test_paths(data_paths_40s,0.4,0.3)
train_paths_50s, validation_paths_50s, test_paths_50s = Make_train_validation_test_paths(data_paths_50s,0.4,0.3)
train_paths_60s, validation_paths_60s, test_paths_60s = Make_train_validation_test_paths(data_paths_60s,0.4,0.3)
train_paths_70s, validation_paths_70s, test_paths_70s = Make_train_validation_test_paths(data_paths_70s,0.4,0.3)
train_paths_80s, validation_paths_80s, test_paths_80s = Make_train_validation_test_paths(data_paths_80s,0.4,0.3)
train_paths_90s, validation_paths_90s, test_paths_90s = Make_train_validation_test_paths(data_paths_90s,0.4,0.3)

In [ ]:
train_paths      = train_paths_10s + train_paths_20s + train_paths_30s + train_paths_40s + train_paths_50s + train_paths_60s + train_paths_70s + train_paths_80s + train_paths_90s
validation_paths = validation_paths_10s + validation_paths_20s + validation_paths_30s + validation_paths_40s + validation_paths_50s + validation_paths_60s + validation_paths_70s + validation_paths_80s + validation_paths_90s
test_paths       = test_paths_10s + test_paths_20s + test_paths_30s + test_paths_40s + test_paths_50s + test_paths_60s + test_paths_70s + test_paths_80s + test_paths_90s
len(train_paths), len(validation_paths), len(test_paths)

# Custom Dataset

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, data_paths, transform=None):
        self.data_paths = data_paths
        self.transform = transform

    def __getitem__(self, idx):
          
        """get data"""
        tempt = self.data_paths[idx] 
        path  = label_data[tempt]['data_path']
        data = np.load(file = path)
        
        
        """get label"""
        
        if label_data[tempt]['age'] <= 40:
            label = 0
        else:
            label = 1
        

        return data,label
    
    
    def __len__(self):
        return len(self.data_paths)

In [ ]:
train_dataset      = CustomDataset(train_paths,transforms.Compose([transforms.ToTensor()]))
validation_dataset = CustomDataset(validation_paths,transforms.Compose([transforms.ToTensor()]))
test_dataset       = CustomDataset(test_paths,transforms.Compose([transforms.ToTensor()]))

print(len(train_dataset),len(validation_dataset), len(test_dataset))

# Data Loader

In [ ]:
"""Train"""
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size = batch_size,
    #sampler = train_sampler,
    shuffle = True, 
    **kwargs
)

"""Validation"""
validation_loader = torch.utils.data.DataLoader(
    dataset=validation_dataset,
    batch_size = batch_size,
    #sampler = valid_sampler,
    shuffle = True,
    **kwargs
)

"""Test"""
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size = test_batch_size,
    shuffle = True,
    **kwargs
)


In [ ]:
print("Length of the train_loader:", len(train_loader))
print("Length of the val_loader:", len(validation_loader))
print("Length of the test_loader:", len(test_loader))

# Optimizer

In [ ]:
model = ECG_CNN().to(device)


optimizer = optim.Adam(model.parameters(), lr=lr)

# Train

## init

In [ ]:
"""Train"""
train_losses        = []
avg_train_losses    = []
Train_baths_ACC     = [] 
Train_ACC           = [] 


"""Validaion"""
valid_losses        = []
avg_valid_losses    = []
Validation_ACC      = []
Valid_ACC_per_Class = []


In [ ]:
num_classes = 2
criterion = nn.CrossEntropyLoss()

In [ ]:
"""save best model"""
best_acc = 0
best_model_save_path = 'path'
best_model_save_path = best_model_save_path + str(experiment_num)

In [ ]:
for epoch in range(1, epochs + 1):
    
    """Train"""
    model.train()
    
    train_loss = 0
    
    for batch_idx, (data, target) in enumerate(train_loader):
        
        
        data, target = data.to(device, dtype=torch.float), target.to(device, dtype=torch.long)
        optimizer.zero_grad()
        
        output = model(data)
        
        #print(output)

        """pred(Cross Entropy)"""
        #pred = output.argmax(dim=1, keepdim=True)  #cross entropy
        pred = F.softmax(output,dim =1).argmax(dim=1, keepdim=True)
        
        """loss"""
        loss = criterion(output, target)
        train_loss += loss.item()

        loss.backward() 
        optimizer.step()
        
        
        correct = 0
        total = target.size(0)
        correct += pred.eq(target.view_as(pred)).sum().item()
        accuracy = 100. * correct / total
        Train_baths_ACC.append(accuracy)
        
        
        if batch_idx % log_interval == 0:
            #1.
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

            #2.
            print('Train set:  batch loss: {:.4f}, Accuracy: {:.0f}% '.format(
                loss.item() ,accuracy))

    
    """Validation"""
    model.eval()
    
    valid_loss = 0
    correct = 0
    total = len(validation_loader.dataset)


    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(validation_loader):
            data, target = data.to(device, dtype=torch.float), target.to(device, dtype=torch.long)
            
            output = model(data)


            """pred(Cross Entropy)"""
            #pred = output.argmax(dim=1, keepdim=True) # cross entropy
            pred = F.softmax(output,dim =1).argmax(dim=1, keepdim=True) 
            
            """loss"""
            loss = criterion(output, target)
            valid_loss += loss.item()
        
            correct += pred.eq(target.view_as(pred)).sum().item()
            

                
    """Loss and ACC """
    
    train_loss /= len(train_loader)
    valid_loss /= len(validation_loader)
    avg_train_losses.append(train_loss)
    avg_valid_losses.append(valid_loss)
    
    Train_ACC.append(sum(Train_baths_ACC)/len(Train_baths_ACC))
    Train_baths_ACC = []
    
    valid_accuracy = 100. * correct / total
    Validation_ACC.append(valid_accuracy)

    
    print('------------------------------------------')
    print('Valid set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        valid_loss, correct, total, valid_accuracy))
    print('-------------------------------------------')
    
    
    """Save best model"""
    
    if valid_accuracy > best_acc:
        torch.save(model, best_model_save_path)
        print("model saved.")
        print('-------------------------------------------')
        best_acc = valid_accuracy
    
    

# Model save and Load

In [ ]:
save_path = 'path'

In [ ]:
save_path = save_path + str(experiment_num)

In [ ]:
torch.save(model, save_path)

# Load best model

In [ ]:
model = torch.load(best_model_save_path)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
model.eval()

# Youden index's cut off 

In [ ]:
true_labels  = np.array([]) 
pred_labels  = np.array([]) 
target_score = np.array([]) 

"""Validation"""
model.eval()
    
valid_loss = 0
correct = 0
total = len(validation_loader.dataset)
    

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(validation_loader):
        data, target = data.to(device, dtype=torch.float), target.to(device, dtype=torch.long)
        output = model(data)
        
        """pred(Cross Entropy)"""     
        output = F.softmax(output,dim=1)
        pred   = output.argmax(dim=1, keepdim=True)
        
        correct += pred.eq(target.view_as(pred)).sum().item()
        

        true_labels = np.append(true_labels,np.array(target.cpu()))
        pred_labels = np.append(pred_labels,np.array(pred.cpu()))  
        target_score = np.append(target_score,np.array(output[:,1].cpu())) 
        
        print('validation: [{}/{}] '.format(batch_idx,len(validation_loader)-1))

In [ ]:
"""Youden’s J statistic. / J = Sensitivity + Specificity – 1"""

# calculate roc curves
FPR, TPR, thresholds = roc_curve(true_labels, target_score)

# get the best threshold
J = TPR - FPR
idx = argmax(J)
best_thresh = thresholds[idx]

print('Best Threshold=%f, sensitivity = %.3f, specificity = %.3f, J=%.3f' % (best_thresh, TPR[idx], 1-FPR[idx], J[idx]))

# Test

In [ ]:
true_labels  = np.array([]) 
pred_labels  = np.array([]) 
target_score = np.array([]) 

"""test"""
model.eval()
    
test_loss = 0
correct = 0
total = len(test_loader.dataset)



with torch.no_grad():
    for batch_idx, (data, target) in enumerate(test_loader):
        
        
        data, target = data.to(device, dtype=torch.float), target.to(device, dtype=torch.long)
        output = model(data)
        
        """pred"""
        output = F.softmax(output,dim=1)
        
        pred = (output[:,1] > best_thresh).int() 
        
        
        """loss"""
        loss = criterion(output, target)
        test_loss += loss.item()
        correct += pred.eq(target.view_as(pred)).sum().item()
        
        true_labels = np.append(true_labels,np.array(target.cpu())) 
        pred_labels = np.append(pred_labels,np.array(pred.cpu()))  
        target_score = np.append(target_score,np.array(output[:,1].cpu())) 
        
        print('test: [{}/{}] '.format(batch_idx,len(test_loader)-1))
        
        
test_loss /= len(test_loader)
test_accuracy = 100. * correct / total


print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
    test_loss, correct, total, test_accuracy))
